In [ ]:
#!/usr/bin/env sage
# Standalone Poseidon implementation and polynomial system extractor
# No external file dependencies

class Poseidon:
    """
    Poseidon hash function for polynomial system extraction
    """
    
    def __init__(self, prime=101, R_F=4, R_P=2, t=4):
        """
        Initialize Poseidon with given parameters
        
        Args:
            prime: Field characteristic
            R_F: Number of full rounds
            R_P: Number of partial rounds  
            t: State size (rate + capacity)
        """
        self.F = GF(prime)
        self.t = t
        self.R_F = R_F
        self.R_P = R_P
        self.prime = prime
        
        # Generate round constants (simplified version)
        self.round_constants = self._generate_round_constants()
        
        # Generate MDS matrix
        self.MDS_matrix = self._generate_mds_matrix()

    def _generate_round_constants(self):
        """Generate round constants using a simple method"""
        import hashlib
        
        num_constants = (self.R_F + self.R_P) * self.t
        constants = []
        
        seed = f"poseidon_{self.prime}_{self.R_F}_{self.R_P}_{self.t}".encode()
        
        for i in range(num_constants):
            hasher = hashlib.sha256(seed + i.to_bytes(4, 'big'))
            const_bytes = hasher.digest()
            const_int = int.from_bytes(const_bytes, 'big')
            constants.append(self.F(const_int))
        
        # Reshape into rounds
        return [constants[i*self.t:(i+1)*self.t] for i in range(self.R_F + self.R_P)]

    def _generate_mds_matrix(self):
        """Generate MDS matrix using Cauchy matrix construction"""
        F = self.F
        t = self.t
        
        # Simple Cauchy matrix: M[i,j] = 1/(i - j + t) where i != j + t
        M = matrix(F, t, t)
        for i in range(t):
            for j in range(t):
                val = i - j + t
                if val == 0:
                    val = 2*t + 1  # Avoid division by zero
                M[i, j] = 1 / F(val)
        
        return M

    def permutation(self, state):
        """Apply Poseidon permutation to state"""
        state = [self.F(s) for s in state]
        F, t, R_F, R_P = self.F, self.t, self.R_F, self.R_P
        MDS = self.MDS_matrix
        constants = self.round_constants
        
        R_f = R_F // 2
        round_counter = 0
        
        # First full rounds
        for r in range(R_f):
            # Add constants
            for i in range(t):
                state[i] += constants[round_counter][i]
            
            # S-box (x^3)
            for i in range(t):
                state[i] = state[i]^3
            
            # Linear layer
            state = list(MDS * vector(state))
            round_counter += 1
        
        # Partial rounds
        for r in range(R_P):
            # Add constants
            for i in range(t):
                state[i] += constants[round_counter][i]
            
            # Partial S-box (only first element)
            state[0] = state[0]^3
            
            # Linear layer  
            state = list(MDS * vector(state))
            round_counter += 1
        
        # Final full rounds
        for r in range(R_f):
            # Add constants
            for i in range(t):
                state[i] += constants[round_counter][i]
            
            # S-box
            for i in range(t):
                state[i] = state[i]^3
            
            # Linear layer
            state = list(MDS * vector(state))
            round_counter += 1
        
        return state

    def hash_single_block(self, input_block):
        """
        Hash a single block using Poseidon permutation
        """
        rate = self.t - 1  # Capacity = 1
        assert len(input_block) <= rate, f"Input block too large: {len(input_block)} > {rate}"
        
        # Initialize state
        state = [self.F(0)] * self.t
        
        # Absorb input
        for i in range(len(input_block)):
            state[i] += self.F(input_block[i])
        
        # Apply permutation
        state = self.permutation(state)
        
        # Return first rate elements
        return state[:rate]


def extract_poseidon_system(prime, R_F, R_P, t, input_block, verbose=True):
    """
    Extract polynomial system for Poseidon preimage attack
    """
    if verbose:
        print(f"=== Poseidon Polynomial System Extraction ===")
        print(f"Prime field: F_{prime}")
        print(f"Full rounds: {R_F}, Partial rounds: {R_P}")
        print(f"State size: {t}")
        print(f"Input block: {input_block}")
    
    # Create Poseidon instance
    poseidon = Poseidon(prime=prime, R_F=R_F, R_P=R_P, t=t)
    
    # Rate calculation
    rate = t - 1  # Capacity = 1
    
    # Create polynomial ring for symbolic input
    num_input_vars = len(input_block)
    ring = PolynomialRing(GF(prime), 'x', num_input_vars)
    input_vars = ring.gens()
    
    if verbose:
        print(f"Rate: {rate}, Capacity: 1")
        print(f"Input variables: {input_vars}")
    
    # Create symbolic computation using polynomial ring as field
    poseidon_symbolic = Poseidon(prime=prime, R_F=R_F, R_P=R_P, t=t)
    
    # Replace field with polynomial ring for symbolic computation
    poseidon_symbolic.F = ring
    poseidon_symbolic.MDS_matrix = matrix(ring, [[ring(poseidon.MDS_matrix[i,j]) for j in range(t)] for i in range(t)])
    poseidon_symbolic.round_constants = [[ring(c) for c in round_const] for round_const in poseidon.round_constants]
    
    # Compute symbolic hash
    symbolic_output = poseidon_symbolic.hash_single_block(input_vars)
    
    if verbose:
        print(f"\n=== Symbolic Output (Hash as polynomials in input variables) ===")
        for i, poly in enumerate(symbolic_output):
            print(f"h_{i} = {poly}")
        
        print(f"\n=== System Statistics ===")
        degrees = [poly.degree() for poly in symbolic_output]
        print(f"Degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total monomials: {sum(len(poly.monomials()) for poly in symbolic_output)}")
    
    return symbolic_output, ring, poseidon


def create_preimage_attack_system(prime, R_F, R_P, t, target_hash, input_size=None, verbose=True):
    """
    Create preimage attack system: given target hash, find input
    """
    if verbose:
        print(f"\n=== Creating Preimage Attack System ===")
    
    # Determine input size
    if input_size is None:
        input_size = t - 1  # Use full rate
    
    # Create dummy input for symbolic system
    dummy_input = list(range(input_size))
    
    # Get symbolic system
    symbolic_output, ring, poseidon = extract_poseidon_system(prime, R_F, R_P, t, dummy_input, verbose=False)
    
    if verbose:
        print(f"Target hash: {target_hash}")
        print(f"Input size: {input_size}")
    
    # Create attack equations: symbolic_output[i] - target_hash[i] = 0
    attack_equations = []
    for i, (symbolic, target) in enumerate(zip(symbolic_output, target_hash)):
        equation = symbolic - target
        attack_equations.append(equation)
        if verbose:
            print(f"Equation {i}: {equation} = 0")
    
    if verbose:
        print(f"\nPreimage attack system: {len(attack_equations)} equations in {input_size} unknowns")
        degrees = [eq.degree() for eq in attack_equations]
        print(f"Equation degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total terms: {sum(len(eq.monomials()) for eq in attack_equations)}")
    
    return attack_equations, ring, poseidon


def solve_preimage_system(attack_equations, ring, verbose=True):
    """
    Attempt to solve the preimage system
    """
    if verbose:
        print(f"\n=== Solving Preimage System ===")
        print(f"Computing Gröbner basis...")
    
    try:
        import time
        start_time = time.time()
        
        # Compute Gröbner basis
        ideal = Ideal(attack_equations)
        gb = ideal.groebner_basis()
        
        end_time = time.time()
        
        if verbose:
            print(f"Gröbner basis computed in {end_time - start_time:.3f} seconds")
            print(f"Gröbner basis has {len(gb)} polynomials")
        
        if gb:
            gb_degrees = [p.degree() for p in gb if p != 0]
            if gb_degrees and verbose:
                print(f"Gröbner basis degrees: {gb_degrees}")
                print(f"Maximum degree in GB: {max(gb_degrees)}")
            
            # Check if system is inconsistent
            if len(gb) == 1 and gb[0] == 1:
                if verbose:
                    print("System is inconsistent (no solutions)")
                return None, gb
            
            # Try to find solutions for small systems
            if len(attack_equations) <= 3:
                try:
                    variety = ideal.variety()
                    if verbose:
                        print(f"Found {len(variety)} solution(s):")
                        for i, sol in enumerate(variety):
                            print(f"  Solution {i}: {sol}")
                    return variety, gb
                except Exception as e:
                    if verbose:
                        print(f"Variety computation failed: {e}")
            else:
                if verbose:
                    print("System too large for variety computation")
            
            return None, gb
        
        else:
            if verbose:
                print("Empty Gröbner basis")
            return None, gb
            
    except Exception as e:
        if verbose:
            print(f"Gröbner basis computation failed: {e}")
        return None, None


def main():
    """
    Main function demonstrating Poseidon polynomial system extraction
    """
    print("Poseidon Polynomial System Extractor")
    print("=" * 50)
    
    # Small parameters for demonstration
    prime = 101
    
    # Test configurations
    configs = [
        {"R_F": 2, "R_P": 1, "t": 4, "name": "Minimal"},
        {"R_F": 1, "R_P": 2, "t": 4, "name": "Small"},  
        {"R_F": 2, "R_P": 2, "t": 4, "name": "Medium"},
    ]
    
    for config in configs:
        print(f"\n{'='*15} {config['name']} Configuration {'='*15}")
        
        R_F, R_P, t = config["R_F"], config["R_P"], config["t"]
        rate = t - 1
        
        # Create test input and compute target hash
        test_input = list(range(rate))
        poseidon = Poseidon(prime=prime, R_F=R_F, R_P=R_P, t=t)
        target_hash = poseidon.hash_single_block(test_input)
        
        # Extract polynomial system
        symbolic_output, ring, _ = extract_poseidon_system(prime, R_F, R_P, t, test_input, verbose=True)
        
        # Create preimage attack system
        attack_equations, ring, _ = create_preimage_attack_system(prime, R_F, R_P, t, target_hash, len(test_input), verbose=True)
        
        # For very small systems, try to solve
        if R_F <= 2 and R_P <= 1:
            solutions, gb = solve_preimage_system(attack_equations, ring, verbose=True)
            
            if solutions:
                print(f"\n=== Preimage Attack Successful! ===")
                if solutions:
                    sol = solutions[0]
                    recovered_input = [sol.get(ring.gen(i), None) for i in range(len(test_input))]
                    print(f"Recovered input: {recovered_input}")
                    print(f"Original input:  {test_input}")
        else:
            print(f"\n=== System extracted successfully ===")
            print(f"System ready for analysis with specialized tools")
    
    print(f"\n=== Summary ===")
    print(f"Successfully demonstrated Poseidon polynomial system extraction")
    print(f"Higher degree systems require specialized algebraic attack tools")


if __name__ == "__main__":
    main()

Poseidon Polynomial System Extractor

=============== Minimal Configuration ===============
=== Poseidon Polynomial System Extraction ===
Prime field: F_101
Full rounds: 2, Partial rounds: 1
State size: 4
Input block: [0, 1, 2]
Rate: 3, Capacity: 1
Input variables: (x0, x1, x2)

=== Symbolic Output (Hash as polynomials in input variables) ===
h_0 = 47*x0^27 - 42*x0^24*x1^3 - 22*x0^21*x1^6 - 46*x0^18*x1^9 + 9*x0^15*x1^12 + 12*x0^12*x1^15 - 23*x0^9*x1^18 - 42*x0^6*x1^21 - 14*x0^3*x1^24 - 32*x1^27 + 38*x0^24*x2^3 + 35*x0^21*x1^3*x2^3 - 5*x0^18*x1^6*x2^3 - 47*x0^15*x1^9*x2^3 - 11*x0^12*x1^12*x2^3 - 5*x0^9*x1^15*x2^3 - 37*x0^6*x1^18*x2^3 + 34*x0^3*x1^21*x2^3 - 28*x1^24*x2^3 + x0^21*x2^6 + 43*x0^18*x1^3*x2^6 - 30*x0^15*x1^6*x2^6 - 33*x0^12*x1^9*x2^6 - 44*x0^9*x1^12*x2^6 - 15*x0^6*x1^15*x2^6 + 27*x0^3*x1^18*x2^6 + 34*x1^21*x2^6 - 29*x0^18*x2^9 - 30*x0^15*x1^3*x2^9 + x0^12*x1^6*x2^9 + 13*x0^9*x1^9*x2^9 + 13*x0^6*x1^12*x2^9 - 20*x0^3*x1^15*x2^9 + 18*x1^18*x2^9 + 14*x0^15*x2^12 + 26*x0^12*x1^3*x